Converting features into ones we can actually use, and moving the target value to the last column.
This can get extended as we add more numerical/categorical features.

Let's make the data (x) and target(Y) and then pickle it.

In [89]:
import numpy as np
import pandas as pd
import julian
from sklearn.preprocessing import OrdinalEncoder
import datetime

In [90]:
df = pd.read_pickle('data_with_target.pkl')


In [91]:
df.head()

,Unnamed: 0,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,...,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape,DISCOVERY_DATETIME,CONT_DATETIME,TIME_TO_CONT
0,0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,511,...,5.0,USFS,CA,63,63.0,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...,2005-02-02 13:00:00,2005-02-03 11:00:00,0 days 22:00:00
1,1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,...,5.0,USFS,CA,61,61.0,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...,2004-05-12 08:45:00,2004-05-13 07:00:00,0 days 22:15:00
2,2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,...,13.0,STATE OR PRIVATE,CA,17,17.0,El Dorado,b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\...,2004-05-31 19:21:00,2004-06-01 16:48:00,0 days 21:27:00
3,3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,...,5.0,USFS,CA,3,3.0,Alpine,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]...,2004-06-28 16:00:00,2004-07-04 04:00:00,5 days 12:00:00
4,4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,...,5.0,USFS,CA,3,3.0,Alpine,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\...,2004-06-28 16:00:00,2004-07-04 00:00:00,5 days 08:00:00


In [92]:
df.columns.values

array(['Unnamed: 0', 'OBJECTID', 'FOD_ID', 'FPA_ID', 'SOURCE_SYSTEM_TYPE',
       'SOURCE_SYSTEM', 'NWCG_REPORTING_AGENCY', 'NWCG_REPORTING_UNIT_ID',
       'NWCG_REPORTING_UNIT_NAME', 'SOURCE_REPORTING_UNIT',
       'SOURCE_REPORTING_UNIT_NAME', 'LOCAL_FIRE_REPORT_ID',
       'LOCAL_INCIDENT_ID', 'FIRE_CODE', 'FIRE_NAME',
       'ICS_209_INCIDENT_NUMBER', 'ICS_209_NAME', 'MTBS_ID',
       'MTBS_FIRE_NAME', 'COMPLEX_NAME', 'FIRE_YEAR', 'DISCOVERY_DATE',
       'DISCOVERY_DOY', 'DISCOVERY_TIME', 'STAT_CAUSE_CODE',
       'STAT_CAUSE_DESCR', 'CONT_DATE', 'CONT_DOY', 'CONT_TIME',
       'FIRE_SIZE', 'FIRE_SIZE_CLASS', 'LATITUDE', 'LONGITUDE',
       'OWNER_CODE', 'OWNER_DESCR', 'STATE', 'COUNTY', 'FIPS_CODE',
       'FIPS_NAME', 'Shape', 'DISCOVERY_DATETIME', 'CONT_DATETIME',
       'TIME_TO_CONT'], dtype=object)

Out of the input values, what we can use as categorical values are probably 'state' and discovery date.  Let's drop everything except for the state (encode to categories) and the discovery month (encode to categories), and save the labels separately.

In [93]:
x = df[['DISCOVERY_DATETIME', 'STATE']]

In [94]:
x.head()


,DISCOVERY_DATETIME,STATE
0,2005-02-02 13:00:00,CA
1,2004-05-12 08:45:00,CA
2,2004-05-31 19:21:00,CA
3,2004-06-28 16:00:00,CA
4,2004-06-28 16:00:00,CA


In [95]:
#Now encode the results:
x['DISCOVERY_DATETIME'].head()


0   2005-02-02 13:00:00
1   2004-05-12 08:45:00
2   2004-05-31 19:21:00
3   2004-06-28 16:00:00
4   2004-06-28 16:00:00
Name: DISCOVERY_DATETIME, dtype: datetime64[ns]

In [96]:
x['DISCOVERY_DATETIME'] = x['DISCOVERY_DATETIME'].apply(lambda x: x.month)


/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [97]:
x.head()

,DISCOVERY_DATETIME,STATE
0,2,CA
1,5,CA
2,5,CA
3,6,CA
4,6,CA


In [98]:
#Encode the motnhs and the states ordinally.
enc = OrdinalEncoder()
st =  enc.fit_transform(np.array(x['STATE']).reshape(-1,1))


In [99]:
enc.categories_

[array(['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
        'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
        'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
        'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN',
        'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY'], dtype=object)]

In [100]:
x['STATE'] = pd.DataFrame(st)

/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [101]:
x.head()

,DISCOVERY_DATETIME,STATE
0,2,4.0
1,5,4.0
2,5,4.0
3,6,4.0
4,6,4.0


In [102]:
#And then let's save it for analysis.
x.to_pickle("wildfire_encoded_data.pkl")

In [103]:
#As for the labels, let's convert those to hours
Y = df['TIME_TO_CONT']
Y.head()

0   0 days 22:00:00
1   0 days 22:15:00
2   0 days 21:27:00
3   5 days 12:00:00
4   5 days 08:00:00
Name: TIME_TO_CONT, dtype: timedelta64[ns]

In [106]:
Y = Y.apply(lambda x: x.total_seconds()/3600)



In [107]:
Y.head()

0     22.00
1     22.25
2     21.45
3    132.00
4    128.00
Name: TIME_TO_CONT, dtype: float64

In [108]:
Y.to_pickle('wildfire_encoded_labels.pkl')